In [9]:
import pandas as pd
import geopandas as gd
import matplotlib.pyplot as plt

df = pd.read_csv(r"C:\Users\PCHOME\Desktop\Jasminehelene\Open_data_dcdj_courses\data-wrangling-and-validation\data\lesson-programmatic\patient-data-anonymisation-exercise.csv")
df.head()

,PATIENT_ID,START,STOP,ENCOUNTERCLASS,DESCRIPTION,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONDESCRIPTION,BIRTHDATE,DEATHDATE,...,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON
0,034e9e3b-2def-4559-bb2a-7850888ae060,2010-01-23T17:45:28Z,2010-01-23T18:10:28Z,ambulatory,Encounter for symptom,129.16,54.16,Acute bronchitis (disorder),14/11/1983,NaN,...,nonhispanic,M,Danvers Massachusetts US,422 Farrell Path Unit 69,Somerville,Massachusetts,Middlesex County,2143.0,42.360697,-71.126531
1,034e9e3b-2def-4559-bb2a-7850888ae060,2012-01-23T17:45:28Z,2012-01-23T18:00:28Z,wellness,General examination of patient (procedure),129.16,129.16,NaN,14/11/1983,NaN,...,nonhispanic,M,Danvers Massachusetts US,422 Farrell Path Unit 69,Somerville,Massachusetts,Middlesex County,2143.0,42.360697,-71.126531
2,034e9e3b-2def-4559-bb2a-7850888ae060,2015-01-26T17:45:28Z,2015-01-26T18:15:28Z,wellness,General examination of patient (procedure),129.16,129.16,NaN,14/11/1983,NaN,...,nonhispanic,M,Danvers Massachusetts US,422 Farrell Path Unit 69,Somerville,Massachusetts,Middlesex County,2143.0,42.360697,-71.126531
3,034e9e3b-2def-4559-bb2a-7850888ae060,2016-12-29T17:45:28Z,2016-12-29T18:00:28Z,ambulatory,Encounter for symptom,129.16,54.16,Acute bronchitis (disorder),14/11/1983,NaN,...,nonhispanic,M,Danvers Massachusetts US,422 Farrell Path Unit 69,Somerville,Massachusetts,Middlesex County,2143.0,42.360697,-71.126531
4,034e9e3b-2def-4559-bb2a-7850888ae060,2017-01-09T17:45:28Z,2017-01-09T18:00:28Z,outpatient,Encounter for check up (procedure),129.16,54.16,NaN,14/11/1983,NaN,...,nonhispanic,M,Danvers Massachusetts US,422 Farrell Path Unit 69,Somerville,Massachusetts,Middlesex County,2143.0,42.360697,-71.126531


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53346 entries, 0 to 53345
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PATIENT_ID         53346 non-null  object 
 1   START              53346 non-null  object 
 2   STOP               53346 non-null  object 
 3   ENCOUNTERCLASS     53346 non-null  object 
 4   DESCRIPTION        53346 non-null  object 
 5   TOTAL_CLAIM_COST   53346 non-null  float64
 6   PAYER_COVERAGE     53346 non-null  float64
 7   REASONDESCRIPTION  13777 non-null  object 
 8   BIRTHDATE          53346 non-null  object 
 9   DEATHDATE          14287 non-null  object 
 10  SSN                53346 non-null  object 
 11  DRIVERS            49285 non-null  object 
 12  PASSPORT           46965 non-null  object 
 13  FIRST              53346 non-null  object 
 14  LAST               53346 non-null  object 
 15  MAIDEN             16597 non-null  object 
 16  MARITAL            436

In [11]:
suppression = ["SSN", "DRIVERS", "PASSPORT", "FIRST", "LAST", "MAIDEN", "ADDRESS", "ZIP"]
# Et les laisser tomber
df.drop(suppression, axis=1, inplace=True)

In [12]:
import uuid

str(uuid.uuid4())

'd539d03a-79be-4be0-b874-8cc3c2232e3d'

In [13]:
import datetime


class Generalization:
    def __init__(self):
        self.days = datetime.timedelta(days=1000)

    def generalize_birth_date(self, birth_date):
        previous_birth_date = datetime.datetime.strptime(birth_date, "%d/%m/%Y")
        new_birth_date = previous_birth_date + self.days
        return new_birth_date.strftime("%d/%m/%Y")

    def reverse_birth_date(self, birth_date):
        previous_birth_date = datetime.datetime.strptime(birth_date, "%d/%m/%Y")
        new_birth_date = previous_birth_date - self.days
        return new_birth_date.strftime("%d/%m/%Y")

    def generalize_death_date(self, death_date):
        if isinstance(death_date, str):
            previous_death_date = datetime.datetime.strptime(death_date, "%d/%m/%Y")
            new_death_date = previous_death_date + self.days
            return new_death_date.strftime("%d/%m/%Y")

    def reverse_death_date(self, death_date):
        if isinstance(death_date, str):
            previous_death_date = datetime.datetime.strptime(death_date, "%d/%m/%Y")
            new_death_date = previous_death_date - self.days
            return new_death_date.strftime("%d/%m/%Y")

    def generalize_date(self, row):
        return [
            row["PATIENT_ID"],
            self.generalize_birth_date(row["BIRTHDATE"]),
            self.generalize_death_date(row["DEATHDATE"])
        ]

    def revert_generalization(self, row):
        return [
            row["PATIENT_ID"],
            self.reverse_birth_date(row["BIRTHDATE"]),
            self.reverse_death_date(row["DEATHDATE"])
        ]



generalization = Generalization()
fields = ["PATIENT_ID", "BIRTHDATE", "DEATHDATE"]


In [14]:
df[fields] = df[fields].apply(generalization.generalize_date, axis=1, result_type="expand")
df.head()

,PATIENT_ID,START,STOP,ENCOUNTERCLASS,DESCRIPTION,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONDESCRIPTION,BIRTHDATE,DEATHDATE,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,CITY,STATE,COUNTY,LAT,LON
0,034e9e3b-2def-4559-bb2a-7850888ae060,2010-01-23T17:45:28Z,2010-01-23T18:10:28Z,ambulatory,Encounter for symptom,129.16,54.16,Acute bronchitis (disorder),10/08/1986,None,M,white,nonhispanic,M,Danvers Massachusetts US,Somerville,Massachusetts,Middlesex County,42.360697,-71.126531
1,034e9e3b-2def-4559-bb2a-7850888ae060,2012-01-23T17:45:28Z,2012-01-23T18:00:28Z,wellness,General examination of patient (procedure),129.16,129.16,NaN,10/08/1986,None,M,white,nonhispanic,M,Danvers Massachusetts US,Somerville,Massachusetts,Middlesex County,42.360697,-71.126531
2,034e9e3b-2def-4559-bb2a-7850888ae060,2015-01-26T17:45:28Z,2015-01-26T18:15:28Z,wellness,General examination of patient (procedure),129.16,129.16,NaN,10/08/1986,None,M,white,nonhispanic,M,Danvers Massachusetts US,Somerville,Massachusetts,Middlesex County,42.360697,-71.126531
3,034e9e3b-2def-4559-bb2a-7850888ae060,2016-12-29T17:45:28Z,2016-12-29T18:00:28Z,ambulatory,Encounter for symptom,129.16,54.16,Acute bronchitis (disorder),10/08/1986,None,M,white,nonhispanic,M,Danvers Massachusetts US,Somerville,Massachusetts,Middlesex County,42.360697,-71.126531
4,034e9e3b-2def-4559-bb2a-7850888ae060,2017-01-09T17:45:28Z,2017-01-09T18:00:28Z,outpatient,Encounter for check up (procedure),129.16,54.16,NaN,10/08/1986,None,M,white,nonhispanic,M,Danvers Massachusetts US,Somerville,Massachusetts,Middlesex County,42.360697,-71.126531
